In [6]:
import os
import numpy as np
import torch
from transformers import BertTokenizer, BertTokenizerFast, BertForSequenceClassification, BertModel
import math
import torch.nn as nn
import os
import numpy as np
import re
import matplotlib
import argparse
# matplotlib.use('Qt5Agg')
# import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import nn
import copy
import math
from tqdm.auto import tqdm,trange
from torch.nn import SyncBatchNorm
from torch.utils.data.dataloader import default_collate
from torch.nn.parallel import DistributedDataParallel
import torch.distributed as dist
from torch.utils.data.distributed import DistributedSampler
# from torch.utils.tensorboard import SummaryWriter
import datasets
import json
import evaluate
from datasets import load_dataset, load_from_disk
work_dir = "./"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 打抽取式标签

In [8]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
dataset

Reusing dataset cnn_dailymail (/home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [29]:
dataset["train"][0]["article"]

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details o

In [30]:
dataset["train"][0]["highlights"]

"Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund ."

In [9]:
from nltk.tokenize import sent_tokenize
sent_tokenize(dataset["train"][0]["article"])

["LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.",
 'Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.',
 '"I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.',
 '"I don\'t think I\'ll be particularly extravagant.',
 '"The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."',
 'At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box off

In [10]:
def split_article(example):
    example["split_article"] = sent_tokenize(example["article"])
    example["split_highlights"] = example["highlights"].split("\n")
    return example

In [11]:
split_dataset = dataset.map(split_article, num_proc=20, remove_columns=["article", "highlights"])

Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-010575512abe7315.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-0653882feb83bddf.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-c91847a1b0512fd3.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-2499040ac1d9181e.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-e715d5e8eeca5c03.arrow


...

Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-91e5f29303ec0bdc.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-3ae59e58f9161ae2.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-e871c17c0bd970fe.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-e0479e57d32d0e3a.arrow


Loading cached processed dataset at /home/liupeng15/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-c388cb399fca2582.arrow


In [12]:
split_dataset["train"][0]["split_article"]

["LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.",
 'Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.',
 '"I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.',
 '"I don\'t think I\'ll be particularly extravagant.',
 '"The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."',
 'At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box off

In [16]:
dataset["train"][1]["split_highlights"]

['Mentally ill inmates in Miami are housed on the "forgotten floor"',
 'Judge Steven Leifman says most are there as a result of "avoidable felonies"',
 'While CNN tours facility, patient shouts: "I am the son of the president"',
 "Leifman says the system is unjust and he's fighting for change ."]

In [13]:
max_article_len = 0
for split_article in tqdm(split_dataset["train"]["split_article"]):
    max_article_len = max(max_article_len, len(split_article))
for split_article in tqdm(split_dataset["validation"]["split_article"]):
    max_article_len = max(max_article_len, len(split_article))
for split_article in tqdm(split_dataset["test"]["split_article"]):
    max_article_len = max(max_article_len, len(split_article))
max_article_len

  0%|          | 0/287113 [00:00<?, ?it/s]

  0%|          | 0/13368 [00:00<?, ?it/s]

  0%|          | 0/11490 [00:00<?, ?it/s]

399

In [14]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')
scores["rouge1"].fmeasure

0.7058823529411765

In [23]:
from rouge_score import rouge_scorer

for target in ['rouge1', 'rouge2', 'rougeL']:
    scorer = rouge_scorer.RougeScorer([target], use_stemmer=True)
    for _ in trange(10000):
        scores = scorer.score('The quick brown fox jumps over the lazy dog', 'The quick brown dog jumps on the log.')


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [6]:
rouge = evaluate.load('rouge')
predictions = ['The quick brown fox jumps over the lazy dog']
references = ['The quick brown dog jumps on the log.']
results = rouge.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.7058823529411765, 'rouge2': 0.26666666666666666, 'rougeL': 0.5882352941176471, 'rougeLsum': 0.5882352941176471}


In [15]:
type(rouge)

evaluate_modules.metrics.evaluate-metric--rouge.b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886.rouge.Rouge

In [16]:
def get_extractive_summary(example):
    extractive_summary_label = [0 for _ in range(max_article_len)]
    extractive_summary_positions = set()
    for i, highlights_sent in enumerate(example["split_highlights"]):
        max_rouge_1 = 0
        max_rouge_1_position = None
        for j, article_sent in enumerate(example["split_article"]):
            if j not in extractive_summary_positions:
#                 rouge_scores = rouge.compute(predictions=[article_sent], references=[highlights_sent])
#                 rouge_1 = rouge_scores["rouge1"] 
                
                rouge_scores = scorer.score(article_sent, highlights_sent)
                rouge_1 = scores["rouge1"].fmeasure
                
                if rouge_1 > max_rouge_1:
                    max_rouge_1 = rouge_1
                    max_rouge_1_position = j
        if max_rouge_1_position is not None:
            extractive_summary_label[max_rouge_1_position] = 1
            extractive_summary_positions.add(max_rouge_1_position)
    example["extractive_summary_label"] = extractive_summary_label
    return example

In [17]:
for _ in trange(100):
    get_extractive_summary(split_dataset["train"][0])


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
rouge_1_dataset = split_dataset.map(get_extractive_summary, num_proc=100)

#0:   0%|          | 0/2872 [00:00<?, ?ex/s]

#1:   0%|          | 0/2872 [00:00<?, ?ex/s]

#2:   0%|          | 0/2872 [00:00<?, ?ex/s]

#3:   0%|          | 0/2872 [00:00<?, ?ex/s]

#4:   0%|          | 0/2872 [00:00<?, ?ex/s]

...

#16:   0%|          | 0/2871 [00:00<?, ?ex/s]

#15:   0%|          | 0/2871 [00:00<?, ?ex/s]

#17:   0%|          | 0/2871 [00:00<?, ?ex/s]

#18:   0%|          | 0/2871 [00:00<?, ?ex/s]

#19:   0%|          | 0/2871 [00:00<?, ?ex/s]

In [24]:
item = rouge_1_dataset["train"][0]
for i, sent in enumerate(item["split_article"]):
    if item["extractive_summary_label"][i] == 1:
        print("*" * 117)
    print(i, sent)
    if item["extractive_summary_label"][i] == 1:
        print("*" * 117)
for i, sent in enumerate(item["split_highlights"]):
    print(i, sent)

*********************************************************************************************************************
0 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
*********************************************************************************************************************
*********************************************************************************************************************
1 Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.
*********************************************************************************************************************
2 "I don't plan to be one of those people who, as soon as they turn 18, s

### 加 cls token

In [7]:
rouge_1_dataset = load_from_disk("./data/hugging_face_cnn_dailymail_rouge_1")
rouge_1_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label'],
        num_rows: 11490
    })
})

In [8]:
def add_cls_sep_token(example):
    example["split_article_cls_join"] = " [SEP] [CLS] ".join(example["split_article"])
    return example

In [9]:
added_cls_dataset = rouge_1_dataset.map(add_cls_sep_token, num_proc=30)

#0:   0%|          | 0/9571 [00:00<?, ?ex/s]

#1:   0%|          | 0/9571 [00:00<?, ?ex/s]

#2:   0%|          | 0/9571 [00:00<?, ?ex/s]

#3:   0%|          | 0/9571 [00:00<?, ?ex/s]

#4:   0%|          | 0/9571 [00:00<?, ?ex/s]

...

#25:   0%|          | 0/383 [00:00<?, ?ex/s]

#26:   0%|          | 0/383 [00:00<?, ?ex/s]

#27:   0%|          | 0/383 [00:00<?, ?ex/s]

#28:   0%|          | 0/383 [00:00<?, ?ex/s]

#29:   0%|          | 0/383 [00:00<?, ?ex/s]

In [10]:
added_cls_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'split_article_cls_join'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'split_article_cls_join'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'split_article_cls_join'],
        num_rows: 11490
    })
})

In [11]:
added_cls_dataset["train"][0]["split_article_cls_join"]

'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. [SEP] [CLS] Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. [SEP] [CLS] "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. [SEP] [CLS] "I don\'t think I\'ll be particularly extravagant. [SEP] [CLS] "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." [SEP] [CLS] At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places belo

In [17]:
max_token_num = 0
for split_article_cls_join in tqdm(added_cls_dataset["train"]["split_article_cls_join"]):
    max_token_num = max(max_token_num, len(split_article_cls_join.split(" ")))
for split_article_cls_join in tqdm(added_cls_dataset["validation"]["split_article_cls_join"]):
    max_token_num = max(max_token_num, len(split_article_cls_join.split(" ")))
for split_article_cls_join in tqdm(added_cls_dataset["test"]["split_article_cls_join"]):
    max_token_num = max(max_token_num, len(split_article_cls_join.split(" ")))
max_token_num

  0%|          | 0/287113 [00:00<?, ?it/s]

  0%|          | 0/13368 [00:00<?, ?it/s]

  0%|          | 0/11490 [00:00<?, ?it/s]

2891

In [30]:
padding_token_length = 3072
max_article_len = 399

In [20]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [21]:
encoded_dataset = added_cls_dataset.map(lambda examples: tokenizer(examples['split_article_cls_join'], truncation=True, padding='max_length', max_length=padding_token_length), num_proc=35, batched=True, remove_columns=["split_article_cls_join"])


#0:   0%|          | 0/9 [00:00<?, ?ba/s]

#2:   0%|          | 0/9 [00:00<?, ?ba/s]

#5:   0%|          | 0/9 [00:00<?, ?ba/s]

#4:   0%|          | 0/9 [00:00<?, ?ba/s]

#3:   0%|          | 0/9 [00:00<?, ?ba/s]

...

#29:   0%|          | 0/1 [00:00<?, ?ba/s]

#32:   0%|          | 0/1 [00:00<?, ?ba/s]

#34:   0%|          | 0/1 [00:00<?, ?ba/s]

#33:   0%|          | 0/1 [00:00<?, ?ba/s]

#31:   0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 11490
    })
})

In [24]:
len(encoded_dataset["train"][0]["input_ids"]), len(encoded_dataset["train"][0]["token_type_ids"]), len(encoded_dataset["train"][0]["attention_mask"])


(3072, 3072, 3072)

In [28]:
cls_id = tokenizer.convert_tokens_to_ids("[CLS]")
print(cls_id)


def cal_clss_segs(example):
    clss = [i for i, x in enumerate(example["input_ids"]) if x == cls_id]
    clss += [-1] * (max_article_len - len(clss))
    seg_flag = 0
    segs = []
    for i in range(1, len(clss)):
        segs += [seg_flag] * (clss[i] - clss[i - 1])
        seg_flag = seg_flag ^ 1
    segs += [seg_flag] * (padding_token_length - len(segs))
    return {"clss": clss, "token_type_ids": segs}

101


In [31]:
clss_segs_dataset = encoded_dataset.map(cal_clss_segs, num_proc=35, batched=False, remove_columns=[])

#0:   0%|          | 0/8204 [00:00<?, ?ex/s]

#1:   0%|          | 0/8204 [00:00<?, ?ex/s]

#2:   0%|          | 0/8204 [00:00<?, ?ex/s]

#3:   0%|          | 0/8204 [00:00<?, ?ex/s]

#4:   0%|          | 0/8204 [00:00<?, ?ex/s]

...

#30:   0%|          | 0/328 [00:00<?, ?ex/s]

#31:   0%|          | 0/328 [00:00<?, ?ex/s]

#32:   0%|          | 0/328 [00:00<?, ?ex/s]

#33:   0%|          | 0/328 [00:00<?, ?ex/s]

#34:   0%|          | 0/328 [00:00<?, ?ex/s]

In [32]:
clss_segs_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'input_ids', 'token_type_ids', 'attention_mask', 'clss'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'input_ids', 'token_type_ids', 'attention_mask', 'clss'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['id', 'split_article', 'split_highlights', 'extractive_summary_label', 'input_ids', 'token_type_ids', 'attention_mask', 'clss'],
        num_rows: 11490
    })
})

In [33]:
clss_segs_dataset["train"][0]["token_type_ids"][52:].index(0)

53

In [34]:
print(clss_segs_dataset["train"][0]["input_ids"][105])

101


In [37]:
def add_label_mask(example):
    value_len = len(example["split_article"])
    padding_len = max_article_len - value_len
    clss_mask = ([False] * value_len) + ([True] * padding_len)
    labels_mask = ([1] * value_len) + ([0] * padding_len)
    
    return {"clss_mask": clss_mask, "labels_mask": torch.tensor(labels_mask)}

In [38]:
final_dataset = clss_segs_dataset.map(add_label_mask, num_proc=35, batched=False, remove_columns=["id", "split_article", "split_highlights"])
final_dataset.save_to_disk("data/hugging_face_cnn_dailymail_rouge_1_for_train")


#0:   0%|          | 0/8204 [00:00<?, ?ex/s]

#1:   0%|          | 0/8204 [00:00<?, ?ex/s]

#2:   0%|          | 0/8204 [00:00<?, ?ex/s]

#3:   0%|          | 0/8204 [00:00<?, ?ex/s]

#4:   0%|          | 0/8204 [00:00<?, ?ex/s]

...

#33:   0%|          | 0/328 [00:00<?, ?ex/s]

#34:   0%|          | 0/328 [00:00<?, ?ex/s]

Saving the dataset (0/17 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

In [66]:
1